In [ ]:
%%bash
pip install instructure-dap-client

In [ ]:
from datetime import datetime
import subprocess
import multiprocessing

def run_bash_command(command):
    subprocess.run(command, shell=True)

api_url = "https://api-gateway.instructure.com"
client_id = "us-west-2#8cb98924-70a9-49b1-bfe9-393e6ae111d5"
client_secret = "3zlv-vTSxyZvnR7grsv56PYBnAgy_N4qug1DP-YIzQs"

# define tables to be extracted from Canvas
tables = [
    "users"
]

commands = []

# create string timestamp to serve as directory
now = datetime.now()
date_ts = now.strftime("%Y%m%d%H%M%S")

# build bash command for the snapshot of each table
for table in tables:

    # build the command string
    file_path = f"/lakehouse/default/Files/Canvas/{table}/{date_ts}"
    commands.append(f"dap --base-url {api_url} --client-id {client_id} --client-secret {client_secret} snapshot --format parquet --output-directory {file_path} --table {table} --namespace canvas")


# create and start a separate process for each command
processes = [multiprocessing.Process(target=run_bash_command, args=(cmd,)) for cmd in commands]

for process in processes:
    process.start()

# wait for all processes to complete
for process in processes:
    process.join()

In [ ]:
# create a delta table for each snapshot dataframe
for table in tables:

    # transform the incremental data and store to a temporary table
    df = spark.read.parquet(f"Files/Canvas/{table}/{date_ts}/*/*.parquet")
    flat_df = df.select("key.*", "value.*", "meta.*")
    flat_df.write.format("delta").mode("overwrite").saveAsTable(f"{table}")
